# Image Classification using Convolution Neural Network
Multi-Label Image Classification Model (1 input - multiple outputs)

# I. Setup and load data

### install dependency and setup

!pip install opencv-python #to remove bullshit image

!pip install tensorflow 

!pip install tensorflow-macos tensorflow-metal #these two packages allow tensorflow to run on MacOS

install miniforge in terminal using https://www.youtube.com/watch?v=w2qlou7n7MA (for mac users)

In mac, we can't use GPU so we need to install miniforge

In [1]:
!python --version

Python 3.10.14


Don't use version above this since imghdr package I'm using might be removed

### import dependencies

In [2]:
import tensorflow as tf
import os # to navigate through file structure
from tensorflow.keras.models import Sequential 
from tensorflow.keras import layers # for data Augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from collections import Counter
from keras.utils import to_categorical
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

tf.__version__

'2.16.1'

In [3]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU') #grab all available gpus on our machine
for gpu in gpus: # limit the memory growth
    tf.config.experimental.set_memory_growth(gpu, True)

- avoid OOM errors
- limit the tenserflow so it won't use all of the vram on our gpu when loading data

In [4]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
IMG_WIDTH = 256
IMG_HEIGHT = 256
batch_size = 32

### remove dogy image
remove corrupted, mislable image

In [6]:
import cv2
# import imghdr

In [7]:
data_dir = 'Furniture_Data_copy'

- First, manually delete any image that has the size below 10 KB (it's too small)
- Next, remove dodgy images using loops

In [8]:
# image_exts = ['jpeg','jpg', 'bmp', 'png']

In [9]:
# image_count = 0 # initialize counter
# import os

# for image_class in os.listdir(data_dir): # loop through each folder and their sub-folder to count using os
#     class_path = os.path.join(data_dir, image_class)
#     if os.path.isdir(class_path):  # check if it's a directory
#         for image_sub in os.listdir(class_path):
#             sub_path = os.path.join(class_path, image_sub)
#             if os.path.isdir(sub_path):  # check if it's a directory
#                 for image in os.listdir(sub_path):
#                     image_path = os.path.join(sub_path,image)
#                     # print(image)
#                     image_count += 1
            
# print("Total images:", image_count) 

In [10]:
# for image_class in os.listdir(data_dir): # loop through each folder and their sub-folder to count using os
#     class_path = os.path.join(data_dir, image_class)
#     if os.path.isdir(class_path):  # check if it's a directory
#         for image_sub in os.listdir(class_path):
#             sub_path = os.path.join(class_path, image_sub)
#             if os.path.isdir(sub_path):  # check if it's a directory
#                 for image in os.listdir(sub_path):
#                     image_path = os.path.join(sub_path,image)
#                     try: 
#                         img = cv2.imread(image_path) # read image information
#                         tip = imghdr.what(image_path)
#                         if tip not in image_exts: 
#                             print('Image not in ext list {}'.format(image_path))
#                             #os.remove(image_path)
#                     except Exception as e: 
#                         print('Issue with image {}'.format(image_path))
#                         # os.remove(image_path)

There is no dodgy image to remove

In [11]:
img = cv2.imread(os.path.join(data_dir,'beds','Asian','2537asian-platform-beds.jpg'))
img.shape

(224, 224, 3)

This show the information of the image: 224 pixels high, 224 pixels wide, 3 = color)

### Load + Preprocessing data
For such a large dataset, an on-the-fly loading mechanism is recommanded rather than loading all data into memory at once. 

ImageDataGenerator is a input pipeline, you can use it to feed data directly to the model without storing huge ass data to the memory

https://viblo.asia/p/xay-dung-input-pipeline-cho-du-lieu-dang-anh-voi-tensoflow-keras-maGK7GnOKj2

create data pipeline (load + preprocess) 

In [12]:
train_datagen = ImageDataGenerator( # template to call data (preprocessing)
    rescale=1./255, #scaling
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.4) # set validation split

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_HEIGHT,IMG_WIDTH),
    batch_size=batch_size,
    class_mode='categorical', # this parameter affect the label(output) shape
    subset='training') # set as training data


# Calculate class weights
counter = Counter(train_generator.classes)
max_val = float(max(counter.values()))
class_weights = {class_id: max_val/num_images for class_id, num_images in counter.items()}


validation_generator = train_datagen.flow_from_directory(
    data_dir, # same directory as training data
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data



Found 21901 images belonging to 3 classes.
Found 14600 images belonging to 3 classes.


In [13]:
first_item = train_generator.__getitem__(0) #check the first item
first_item[1].shape

(32, 3)

### Data distribution

In [14]:
# # Get all labels from the dataset
# all_labels = []
# for _, labels in train_generator:
#     all_labels.extend(labels)

# # Convert to numpy array for manipulation
# all_labels = np.array(all_labels)

# # Count the unique labels and their occurrences
# unique_labels, counts = np.unique(all_labels, return_counts=True)

# # Create a dictionary to map labels to class names
# label_dict = {0: 'bed (0)', 1: 'chair (1)', 2: 'dresser (2)'}

# # Replace labels with class names
# class_names = [label_dict[label] for label in unique_labels]

# # Plotting the distribution
# plt.figure(figsize=(10, 5))

# # Use a color palette from seaborn
# palette = sns.color_palette("hls", len(unique_labels))

# bars = plt.bar(class_names, counts, color=palette)

# # Add counts on top of each bar
# for bar in bars:
#     yval = bar.get_height()
#     plt.text(bar.get_x() + bar.get_width()/2, yval + 0.05, yval, ha='center', va='bottom')

# plt.xlabel('Class')
# plt.ylabel('Count')
# plt.title('Distribution of Classes')
# plt.show()


In [15]:
# # Get all labels from the dataset
# all_labels = []
# for _, labels in data:
#     all_labels.extend(labels.numpy())

# # Convert to numpy array for manipulation
# all_labels = np.array(all_labels)

# # Count the unique labels and their occurrences
# unique_labels, counts = np.unique(all_labels, return_counts=True)

# # Create a dictionary to map labels to class names
# label_dict = {0: 'bed (0)', 1: 'chair (1)', 2: 'dresser (2)'}

# # Replace labels with class names
# class_names = [label_dict[label] for label in unique_labels]

# # Plotting the distribution
# plt.figure(figsize=(10, 5))

# # Use a color palette from seaborn
# palette = sns.color_palette("hls", len(unique_labels))

# bars = plt.bar(class_names, counts, color=palette)

# # Add counts on top of each bar
# for bar in bars:
#     yval = bar.get_height()
#     plt.text(bar.get_x() + bar.get_width()/2, yval + 0.05, yval, ha='center', va='bottom')

# plt.xlabel('Class')
# plt.ylabel('Count')
# plt.title('Distribution of Classes')
# plt.show()


The classes are highly imbalanced

**Solution:** I will use the **class_weights** argument in model.fit (make the model learn more from the minority class).

**Other sugestions:**
- Data augmentation:  make the most of the minority class (I can't find a way to do it with image_dataset_from_directory or flow_from_directory)
- Oversampling: they are bad for image data since it require images to be reshaped into 2D which would lose its information

https://stackoverflow.com/questions/53666759/use-smote-to-oversample-image-data

https://stackoverflow.com/questions/41648129/balancing-an-imbalanced-dataset-with-keras-image-generator

https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

## Data Augmentation
 
 To avoid data leak, data augmentation should be performed before splitting

test the Data Augmentation

In [16]:
# #Augmenting the images
# data_augmentation = tf.keras.Sequential(
#     [
#         tf.keras.layers.RandomFlip("horizontal"),
#         tf.keras.layers.RandomRotation(0.1),
#         tf.keras.layers.RandomZoom(0.2)
#     ]
# )

# #test the Data Augmentation on an image in the dataset
# plt.figure(figsize=(10, 10))
# for images, _ in data.take(2): # take an image
#     for i in range(9): # make 9 versions of it
#         augmented_images = data_augmentation(images)
        
#         ax = plt.subplot(3, 3, i + 1)# plot
#         plt.imshow(augmented_images[0].numpy().astype("uint8"))
        

 we will "augment" them via a number of random transformations, so that our model would never see twice the exact same picture. This helps prevent overfitting and helps the model generalize better.

In [17]:


# datagen = ImageDataGenerator(
#         rotation_range=40,
#         width_shift_range=0.2,
#         height_shift_range=0.2,
#         rescale=1./255,
#         shear_range=0.2,
#         zoom_range=0.2,
#         horizontal_flip=True,
#         fill_mode='nearest')

In [18]:
#train_cw = tf.data.Dataset.from_tensor_slices((images, labels, class_weight=='balanced'))

### split data

In [19]:
#len(data)

we have 1141 batches, each of them contain 32 images (in total there are 36502 images of 3 classes)

decide the amount of train, val, test

In [20]:
# train_size = int(len(data)*.6)
# val_size = int(len(data)*.2)
# test_size = int(len(data)*.2)

we gonna use "take" and "skip" in tensorflow to split whole dataset into 3

In [21]:
# train = scaled_data.take(train_size) # data already has x,y seperately
# val = scaled_data.skip(train_size).take(val_size)
# test = scaled_data.skip(train_size+val_size).take(test_size)

In [22]:
# trainlen = len(train)
# vallen = len(val)
# testlen = len(test)
# print ("number of patches in train data: ", trainlen)
# print ("number of patches in val data: ", vallen)
# print ("number of patches in test data: ", testlen)
# print ("\neach patch has contains 32 images")

Then we create batch to access to x,y of each sub-dataset

# Model

In [23]:
from tensorflow.keras.models import Sequential # the model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout #layer of that model
from sklearn.utils import class_weight

In [24]:
# model = Sequential() # define a model 

add layers to the model

In [25]:
# # Convolutional layers
# model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(IMG_WIDTH,IMG_HEIGHT,3))) # specify the input shape in the first layer
# model.add(MaxPooling2D()) 
# model.add(Conv2D(32, (3,3), 1, activation='relu'))
# model.add(MaxPooling2D())
# model.add(Conv2D(16, (3,3), 1, activation='relu'))
# model.add(MaxPooling2D())

# # Flatten layer to transition from convolutional to fully connected layers
# model.add(Flatten())

# # Fully connected layers
# model.add(Dense(256, activation='relu'))
# # Output (3 category = 3 outputs)
# model.add(Dense(3, activation='softmax')) # for multiclass classification, use Softmax

In [26]:
# model.compile('adam', loss=tf.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

Use "SparseCategoricalCrossentropy()" crossentropy loss function when:
- there are two or more label classes.
- labels are provided as integers

In [27]:
# model.summary()

# Train

In [28]:
# logdir='logs' # create a logs folder (for check point)

In [29]:
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir) # (for check point)

**handle imbalance** by class weight

In [30]:
# histCNN = model.fit(train_generator, 
#                     epochs=1, 
#                     validation_data=validation_generator,
#                     class_weight = class_weights, 
#                     callbacks=[tensorboard_callback])

## Duong's model

In [31]:
from tensorflow import keras
from keras import layers
from keras import Model
from keras.models import Sequential
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dropout, Dense, BatchNormalization, Activation, Flatten
from keras.optimizers import Adam
from keras.applications.vgg19 import VGG19
import cv2
from sklearn.preprocessing import LabelEncoder

In [32]:
# Load the pretained model
pretrained_model = VGG19(
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
    include_top=False,
    weights='imagenet',
    pooling='max'
)

pretrained_model.trainable = False

2024-05-10 09:16:18.644868: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-05-10 09:16:18.644901: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-05-10 09:16:18.644907: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-05-10 09:16:18.644926: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-05-10 09:16:18.644942: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [33]:
model_path = './Models/vgg19-model.weights.h5'
model_callback = ModelCheckpoint(model_path,
                                 save_weights_only=True,
                                 monitor="val_accuracy",
                                 save_best_only=True)

early_stopping = EarlyStopping(monitor = "val_loss", # watch the val loss metric
                               patience = 5,
                               restore_best_weights = True) # if val loss decreases for 3 epochs in a row, stop training

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-10)

tensor_board = TensorBoard(log_dir='./logs')

In [34]:
inputs = pretrained_model.input

x = Dense(128, activation='relu')(pretrained_model.output)
x = BatchNormalization()(x)
x = Dropout(0.45)(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.45)(x)


outputs = Dense(3, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer=Adam(0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

historyDuong = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    epochs=1,
    callbacks=[
        early_stopping,
        tensor_board,
        model_callback,
        reduce_lr
    ]
)

2024-05-10 09:16:20.816274: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
/Users/quangdong/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  5/685 ━━━━━━━━━━━━━━━━━━━━ 23:43 2s/step - accuracy: 0.3935 - loss: 1.8738

KeyboardInterrupt: 

In [ ]:
#hhist = model.fit(train, epochs=1, validation_data=val, sample_weight='balanced', callbacks=[tensorboard_callback])

In [ ]:
# fig = plt.figure()
# plt.plot(histCNN.history['accuracy'], color='teal', label='accuracy')
# plt.plot(histCNN.history['val_accuracy'], color='orange', label='val_accuracy')
# fig.suptitle('Accuracy', fontsize=20)
# plt.legend(loc="upper left")
# plt.show()

In [ ]:
	
# fig = plt.figure()
# plt.plot(hist.history['loss'], color='teal', label='loss')
# plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
# fig.suptitle('Loss', fontsize=20)
# plt.legend(loc="upper left")
# plt.show()

In [ ]:
# from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
# pre = Precision()
# re = Recall()
# acc = BinaryAccuracy()

In [ ]:
# for batch in test.as_numpy_iterator(): 
#     X, y = batch
#     yhat = model.predict(X)
#     pre.update_state(y, yhat)
#     re.update_state(y, yhat)
#     acc.update_state(y, yhat)

In [ ]:
# print(pre.result(), re.result(), acc.result())

## Hyper parameters tunning

In [ ]:
#definr some Hyper parameters
AUTO = tf.data.AUTOTUNE
IMG_SIZE = 28
BATCH_SIZE = 128
EPOCHS = 100

# Test

let pick a random image on the internet to test (unseen data)

In [ ]:
# img = cv2.imread('Screenshot 2024-05-09 at 11.41.28.PNG')
# plt.imshow(img)
# plt.show()

resize that image so it could fit into the model

In [ ]:
# resize = tf.image.resize(img, (256,256))
# plt.imshow(resize.numpy().astype(int))
# plt.show()

In [ ]:
# yhat = model.predict(np.expand_dims(resize/255, 0)) 

np.expand_dims to put the image into an extra array (model doesn't work with 1 image)

In [ ]:
# yhat

The images are now load, shuffel, resize, set batch size = 32 and label:
- 0 is bed
- 1 is chair
- 2 is dresser
- 3 is lamp and 
- 4 is sofa
- 5 is table

# save model

In [ ]:
from tensorflow.keras.models import load_model